In [6]:
import spacy
import pandas as pd
from spacy import displacy
import os
import re

In [7]:
# Cargar el archivo CSV con los argumentos
df = pd.read_csv(r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\limpios\argumentos_procesados.csv")
print(df.head())
df.info()

               titulo                                      contenido_arg
0            Clitofon  ,Clitofon, acusado por Sócrates de haber censu...
1               Lisis  ,El objeto de este diálogo es la Amistad, títu...
2     Timeo de Locres  ,Timeo, nacido entre los locrios epicefirianos...
3               Fedro  ,Según una tradición, que no tenemos necesidad...
4  segundo Alcibíades  ,No debe orarse ligeramente. Dirigir a los dio...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   titulo         16 non-null     object
 1   contenido_arg  16 non-null     object
dtypes: object(2)
memory usage: 388.0+ bytes


In [8]:
# Función para limpiar texto eliminando referencias numéricas en corchetes
def limpiar_texto(texto):
    return re.sub(r'\s*\[\d+\]', '', texto)

In [9]:
# Carga modelo español large
nlp = spacy.load('es_core_news_lg')

c:\Users\Lucas\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'es_core_news_lg' (3.5.0) was trained with spaCy v3.5.0 and may not be 100% compatible with the current version (3.8.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [10]:
"""
Extracción de premisas y conclusiones:
Identificamos patrones de premisas ("porque", "ya que", "dado que") 
"""
# Función para extraer las premisas de los textos
def extraer_premisas(texto):
    doc = nlp(texto)
    premisas = [sent.text for sent in doc.sents if any(p in sent.text.lower() for p in ["porque", "ya que", "dado que"])]
    return {"premisas": premisas}

In [11]:
# Función para extraer las conclusiones de los textos
def extraer_conclusiones(texto):
    doc = nlp(texto)
    conclusiones = [sent.text for sent in doc.sents if any(c in sent.text.lower() for c in ["por lo tanto", "así que", "en consecuencia"])]
    return {"conclusiones": conclusiones}

In [12]:
# Función para detectar contradicciones en los textos
def detectar_contradicciones(texto):
    doc = nlp(texto)
    contradicciones = [sent.text for sent in doc.sents if any(word in sent.text.lower() for word in ["sin embargo", "aunque", "no obstante"])]
    return contradicciones

In [13]:
# Aplicar la detección de contradicciones
df["contradicciones"] = df["contenido_arg"].apply(detectar_contradicciones)

# Mostrar contradicciones encontradas
print(df[["titulo", "contradicciones"]].head())

               titulo                                    contradicciones
0            Clitofon                                                 []
1               Lisis  [Sin embargo, el hombre que ama a otro puede n...
2     Timeo de Locres                                                 []
3               Fedro  [,Sin embargo, existe un lazo entre estas dos ...
4  segundo Alcibíades                                                 []


In [14]:
# Función para extraer fragmentos de estilo socrático (preguntas y razonamientos lógicos)
def extraer_fragmentos_socraticos(texto):
    doc = nlp(texto)
    fragmentos = [sent.text for sent in doc.sents if "¿" in sent.text or ":" in sent.text]
    return fragmentos

# Aplicar la extracción de fragmentos socráticos
df["fragmentos_socraticos"] = df["contenido_arg"].apply(extraer_fragmentos_socraticos)

# Mostrar fragmentos encontrados
print(df[["titulo", "fragmentos_socraticos"]].head())

               titulo                              fragmentos_socraticos
0            Clitofon                                                 []
1               Lisis  [,¿Es el amigo el que ama o el que es amado?, ...
2     Timeo de Locres  [Por otra parte, ¿cómo puede admitirse esta id...
3               Fedro  [,¿Cuál es el objeto del diálogo?, ,En el ,pue...
4  segundo Alcibíades  [Lo mejor es fiarse a los dioses mismos para t...


In [15]:
# Función de procesamiento de texto (tokenización, lematización, eliminación de stopwords y puntuación)
def procesar_argumento(texto):
    doc = nlp(texto)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

In [16]:
# Procesar los argumentos
df["argumento_procesado"] = df["contenido_arg"].apply(procesar_argumento)

# Mostrar resultados procesados
print(df[["titulo", "argumento_procesado"]].head())

               titulo                                argumento_procesado
0            Clitofon  clitofon acusado sócrates censurar conversació...
1               Lisis  objeto diálogo amistad título lleno esperanza ...
2     Timeo de Locres  timeo nacido locrio epicefiriano grecia cuyo n...
3               Fedro  tradición necesidad discutir obra juventud pla...
4  segundo Alcibíades  orar él ligeramente dirigir dios súplica pedir...


In [20]:
# Aplicar las funciones de extracción al dataframe
df["premisas"] = df["contenido_arg"].apply(extraer_premisas)
df["conclusiones"] = df["contenido_arg"].apply(extraer_conclusiones)
df["contradicciones"] = df["contenido_arg"].apply(detectar_contradicciones)
df["fragmentos_socraticos"] = df["contenido_arg"].apply(extraer_fragmentos_socraticos)
df["argumento_procesado"] = df["contenido_arg"].apply(procesar_argumento)

In [21]:
# Función para mostrar el contenido estructurado
def mostrar_contenido_estructurado(fila):
    print(f"📚 Título: {fila['titulo']}")
    print("🔹 Premisas:")
    for premisa in fila["premisas"]:
        print(f"  - {premisa}")
    
    print("🔹 Conclusiones:")
    for conclusion in fila["conclusiones"]:
        print(f"  - {conclusion}")
    
    print("🔹 Contradicciones:")
    for contradiccion in fila["contradicciones"]:
        print(f"  - {contradiccion}")
    
    print("🔹 Fragmentos Socráticos:")
    for fragmento in fila["fragmentos_socraticos"]:
        print(f"  - {fragmento}")
    
    print("🔹 Argumento Procesado:")
    print(f"  {fila['argumento_procesado']}")
    print("-" * 60)

In [22]:
# Mostrar la información estructurada para cada fila del dataframe
for index, fila in df.iterrows():
    mostrar_contenido_estructurado(fila)

📚 Título: Clitofon
🔹 Premisas:
  - premisas
🔹 Conclusiones:
  - conclusiones
🔹 Contradicciones:
🔹 Fragmentos Socráticos:
🔹 Argumento Procesado:
  clitofon acusado sócrates censurar conversación filosófico alabar lección sofista trasimaco defender exponer sócrates pensar discurso página resumir palabra sócrates hombre maravilloso exhortar virtud desempeñar noble tarea incurrir error pasar bastar inspirar yo deseo virtuoso preciso enseñar yo ser él prácticamente preciso mostrar camino señalar dificultad obstáculo necesario guíar llegar término ánimo indagar censura justo ser composición escaso valor
------------------------------------------------------------
📚 Título: Lisis
🔹 Premisas:
  - premisas
🔹 Conclusiones:
  - conclusiones
🔹 Contradicciones:
  - Sin embargo, el hombre que ama a otro puede no ser correspondido; más aún, puede ser odioso al que ama, cosa que se ve comúnmente en la vida.
  - Y sin embargo, ¿qué cosas menos amigas, o más bien, qué cosas más enemigas?
  - ,Sócrates, 

In [23]:
# Combinamos todos los datos extraídos en un formato estructurado
def combinar_argumentos(row):
    argumentos = {
        "titulo": row["titulo"],
        "premisas": row.get("premisas", []),
        "conclusiones": row.get("conclusiones", []),
        "contradicciones": row.get("contradicciones", []),
        "fragmentos_socraticos": row.get("fragmentos_socraticos", []),
        "argumento_procesado": row.get("argumento_procesado", "")
    }
    return argumentos

df["argumentos_combinados"] = df.apply(combinar_argumentos, axis=1)

In [24]:
# Guardar los datos procesados en un archivo JSON
output_directory = r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\procesed"
output_file = os.path.join(output_directory, "argumentos_socraticos_procesados.json")

In [25]:
# Guardamos en formato JSON
df["argumentos_combinados"].to_json(output_file, orient="records", lines=True)

